# DLIB ResNet Pretrained Model

In [1]:
import dlib
print(dlib.__version__)


19.22.99


In [2]:



# Initialize dlib's face detector (HOG-based)
# detector = dlib.get_frontal_face_detector()

# Load the shape predictor model

predictor = dlib.shape_predictor(r"E:\My thesis work\shape_predictor_68_face_landmarks.dat")



In [3]:
import cv2
print(cv2.__version__)


4.10.0


In [4]:
import numpy as np
print(np.__version__)


1.26.4


In [5]:
import sys
print(sys.version)


3.10.9 (tags/v3.10.9:1dd9be6, Dec  6 2022, 20:01:21) [MSC v.1934 64 bit (AMD64)]


In [6]:
import torch
print(torch.__version__)


2.2.2+cpu


# Align Faces and Create Face Encodings

In [1]:
import dlib
import cv2
import os
import numpy as np
import json

# Load the face detector and face recognition model from dlib
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(r"F:\Thesis latest work(6.12.24)\My thesis work\PRETRAINED DLIB Resnet\shape_predictor_68_face_landmarks.dat")
facerec = dlib.face_recognition_model_v1(r"F:\Thesis latest work(6.12.24)\My thesis work\PRETRAINED DLIB Resnet\dlib_face_recognition_resnet_model_v1.dat")

def get_face_encodings(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    encodings = []
    for face in faces:
        landmarks = sp(gray, face)
        encoding = facerec.compute_face_descriptor(img, landmarks)
        encodings.append(np.array(encoding))

    return encodings

def prepare_dataset(dataset_path):
    face_encodings = []
    labels = []
    label_to_name = {}  # Dictionary to store label-name mapping
    
    # Track progress by counting processed persons and images
    total_people = len(os.listdir(dataset_path))
    print(f"Starting to process dataset with {total_people} people...")

    for label, person_name in enumerate(os.listdir(dataset_path)):
        person_path = os.path.join(dataset_path, person_name)
        if os.path.isdir(person_path):
            label_to_name[label] = person_name  # Map label to the person's name
            print(f"\nProcessing person {label + 1}/{total_people}: {person_name}")
            for img_name in os.listdir(person_path):
                img_path = os.path.join(person_path, img_name)
                print(f"  Processing image: {img_name}")
                
                encodings = get_face_encodings(img_path)
                for encoding in encodings:
                    face_encodings.append(encoding)
                    labels.append(label)

    print("\nDataset processing completed.")
    
    # Save the label-to-name dictionary as JSON
    with open("label_to_name.json", "w") as f:
        json.dump(label_to_name, f)
    print("Label-to-name mapping saved successfully.")

    return face_encodings, labels

# Path to your dataset folder
dataset_path = r"F:\Thesis latest work(6.12.24)\My thesis work\Essential Folder\Train_Images"
encodings, labels = prepare_dataset(dataset_path)

# Save the encodings and labels for later use
np.save("face_encodings.npy", encodings)
np.save("labels.npy", labels)
print("Encodings and labels saved successfully.")


Starting to process dataset with 69 people...

Processing person 1/69: Abdur_Rahman_E241018
  Processing image: Abdur_Rahman_E241018_10_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_10_aug_3.jpg
  Processing image: Abdur_Rahman_E241018_10_aug_4.jpg
  Processing image: Abdur_Rahman_E241018_11_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_11_aug_3.jpg
  Processing image: Abdur_Rahman_E241018_11_aug_4.jpg
  Processing image: Abdur_Rahman_E241018_12_aug_1.jpg
  Processing image: Abdur_Rahman_E241018_12_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_13_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_13_aug_3.jpg
  Processing image: Abdur_Rahman_E241018_14_aug_1.jpg
  Processing image: Abdur_Rahman_E241018_14_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_14_aug_4.jpg
  Processing image: Abdur_Rahman_E241018_15_aug_1.jpg
  Processing image: Abdur_Rahman_E241018_15_aug_2.jpg
  Processing image: Abdur_Rahman_E241018_15_aug_4.jpg
  Processing image: Abdur_Rahman_E241018_16_

# Train The Model f1,recall,precision,accuracy,training time all here

In [3]:
import time
from sklearn.svm import SVC
import numpy as np
import joblib
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Load the encodings and labels
encodings = np.load("face_encodings.npy")
labels = np.load("labels.npy")

# Start the timer for the entire process
overall_start_time = time.time()

# Split the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(encodings, labels, test_size=0.3, random_state=42)

# Start the timer for training
start_time = time.time()

# Train the SVM classifier
classifier = SVC(kernel="linear", probability=True)
classifier.fit(X_train, y_train)

# End the timer after training
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")

# Start the timer for making predictions
prediction_start_time = time.time()

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# End the timer after predictions
prediction_end_time = time.time()
prediction_time = prediction_end_time - prediction_start_time
print(f"Prediction time: {prediction_time:.2f} seconds")

# Calculate Precision, Recall, F1 Score, Accuracy, and Macro Precision
precision_weighted = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Calculate Macro Precision
precision_macro = precision_score(y_test, y_pred, average='macro')

# Print the metrics
print(f"Precision (Weighted): {precision_weighted:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (Macro): {precision_macro:.4f}")

# End the timer for the overall process
overall_end_time = time.time()
overall_process_time = overall_end_time - overall_start_time
print(f"Total process time: {overall_process_time:.2f} seconds")

# Save the classifier for later use
joblib.dump(classifier, "face_recognition_model.pkl")


Training time: 0.41 seconds
Prediction time: 0.07 seconds
Precision (Weighted): 0.9652
Recall: 0.9778
F1 Score: 0.9706
Accuracy: 0.9778
Precision (Macro): 0.7567
Total process time: 0.49 seconds


C:\Users\Sakur\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Sakur\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


['face_recognition_model.pkl']

# Recognize Face

In [2]:
def recognize_face(frame, classifier, detector, sp, facerec):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = sp(gray, face)
        encoding = facerec.compute_face_descriptor(frame, landmarks)
        encoding = np.array(encoding).reshape(1, -1)
        
        # Use the classifier to predict the label
        prediction = classifier.predict(encoding)
        confidence = classifier.decision_function(encoding)
        
        # Get the label for the predicted person
        label = prediction[0]
        
        # Display the name or ID
        cv2.putText(frame, f"Person ID: {label}", (face.left(), face.top()-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

    return frame

# Load the trained classifier
classifier = joblib.load("face_recognition_model.pkl")

# Initialize video capture (use 0 for the default webcam)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    # Recognize face
    frame = recognize_face(frame, classifier, detector, sp, facerec)
    
    # Show the result
    cv2.imshow("Face Recognition", frame)

    # Exit when the user presses the 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Face Recognition with name

In [1]:
import dlib
import cv2
import numpy as np
import joblib
import json

# Load the face detector and models
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(r"F:\Thesis latest work(6.12.24)\My thesis work\PRETRAINED DLIB Resnet\shape_predictor_68_face_landmarks.dat")
facerec = dlib.face_recognition_model_v1(r"F:\Thesis latest work(6.12.24)\My thesis work\PRETRAINED DLIB Resnet\dlib_face_recognition_resnet_model_v1.dat")
classifier = joblib.load("face_recognition_model.pkl")

# Load the label-to-name dictionary
with open("label_to_name.json", "r") as f:
    label_to_name = json.load(f)

def recognize_face(frame, classifier, detector, sp, facerec, label_to_name):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        landmarks = sp(gray, face)
        encoding = facerec.compute_face_descriptor(frame, landmarks)
        encoding = np.array(encoding).reshape(1, -1)
        
        # Predict the label
        prediction = classifier.predict(encoding)
        label = prediction[0]
        
        # Get the person's name from the label
        name = label_to_name.get(str(label), "Unknown")

        # Display the name on the frame
        cv2.putText(frame, f"Name: {name}", (face.left(), face.top()-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)
        cv2.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 255, 0), 2)

    return frame

# Initialize video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Recognize face
    frame = recognize_face(frame, classifier, detector, sp, facerec, label_to_name)
    
    # Show the result
    cv2.imshow("Face Recognition", frame)

    # Exit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
